# ANÁLISIS DEL CONJUNTO DE DATOS Y EVALUACIÓN DEL MODELO

## 1. Conjuntos de datos

En este apartado, se describen los diferentes conjuntos de datos obtenidos en base a los diálogos. Cada registro está formado por una columna de entrada, que es el texto a clasificar, y una columna de salida, que es la intención en la que se clasifica el texto.

Según se define en el apartado **4. Identificación de requisitos** de la memoria, se van a tratar las siguientes intenciones:

- Registrar saludo
- Registrar toma de medicamento
- Registrar estado emocional
- Registrar cita médica
- Registrar despedida

Los objetivos cubiertos son:
- Detectar las necesidades de las personas mayores: en base al conjunto de datos etiquetado, se listan las necesidades y el número de registros que se han clasificado en cada texto. De este modo, es posible evaluar qué necesidades son las más importantes.
- Conseguir un conjunto de datos de un mínimo de 100 registros: El objetivo es conseguir un conjunto de datos balanceado y sin duplicados de un mínimo de 100 registros de las intenciones que se van a tratar. Utilizamos sólo las intenciones que tienen más registros.
- Conseguir accuracy 85%: lala

In [ ]:
import json
import matplotlib.pyplot as plt
import pandas as pd

def getDataset(version, sufix=""):
    path = f"/home/jovyan/data/dataset/{version}/data{sufix}.json"
    with open(path, 'r') as f:
      data = json.load(f)
    return data

def getDatasetAsDictionaryText(version):
    dataset = getDataset(version)
    datasetDictionary = {}
    for item in dataset:
        datasetDictionary[item["text"]] = item
    return datasetDictionary

def dataToCSV(dataset, version, sufix=""):
    entries = []
    for entry in dataset:
        text = entry["text"]
        intent = entry["intent"]
        entryText = f"{text};{intent}"
        entries.append(entryText)
    path = f"/home/jovyan/data/dataset/{version}/data{sufix}.csv"
    with open(path,'w') as f:
        for line in entries:
            f.write(line)
            f.write('\n')
    
def writeCSV(entries, version, sufix=""):
    path = f"/home/jovyan/data/dataset/{version}/data{sufix}.csv"
    with open(path, 'w') as f:
      json.dump(entries, f)

def loadDataframe(version, sufix=""):
    dataset = getDataset(version, sufix)
    entries = dataToCSV(dataset, version, sufix)
    path = f"/home/jovyan/data/dataset/{version}/data{sufix}.csv"
    df = pd.read_csv(path, names=["text", "intent"], sep=";")
    return df

def filterDataframe(df):
    filteredColumns = ['REGISTRAR_TOMA_MEDICAMENTO', 'REGISTRAR_ESTADO_EMOCIONAL', 'REGISTRAR_CITA_MEDICA', 'REGISTRAR_SALUDO', 'REGISTRAR_DESPEDIDA']
    filter = df['intent'].isin(filteredColumns)
    dfFiltered = df[filter]
    return dfFiltered

def saveTrainTestDataset(version):
    dataset = getDatasetAsDictionaryText(version)
    df = loadDataframe(version)
    train=df.sample(frac=0.8,random_state=2) #random state is a seed value
    test=df.drop(train.index)
    saveDF(version, train, "train", dataset)
    saveDF(version, test, "test", dataset)
    dfTrain = loadDataframe(DATASET_VERSION, "_train")
    dfTest = loadDataframe(DATASET_VERSION, "_test")
    return dfTrain, dfTest
        
def saveDF(version, df, split, dataset):
    output = {}
    for index, row in df.iterrows():
        output[row["text"]] = dataset[row["text"]]

    toJSON = []
               
    for item in output.keys():
        toJSON.append(output[item])
    

    path = f"/home/jovyan/data/dataset/{DATASET_VERSION}/data_{split}.json"
    with open(path, "w", encoding="utf-8") as f:
        json.dump(toJSON, f, ensure_ascii=False, indent=4)
        print(f"Saved at {path}")

In [ ]:
DATASETS = {}
DATATRAIN = {}
DATATEST = {}

### 1.1. Conjunto v1

In [ ]:
DATASET_VERSION = "v2"
LANGUAJE_MODEL = "dialogflow"
# Versión del conjunto de datos a utilizar: v2
%run -i python dataset_generate.py prod "v3"

In [ ]:
DATASET_VERSION = "v1" # Versión del conjunto de datos
df = loadDataframe(DATASET_VERSION)
df.head()

In [ ]:
df.shape

In [ ]:
df["intent"].unique()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df[df['text']=='Hola']

In [ ]:
df['intent'].value_counts()

In [ ]:
dfFiltered = filterDataframe(df)
dfFiltered.head()

In [ ]:
dfFiltered.shape

In [ ]:
dfFiltered['intent'].value_counts()

In [ ]:
dfTrain, dfTest = saveTrainTestDataset(DATASET_VERSION)
DATASETS["v1"] = df
DATATRAIN["v1"] = dfTrain
DATATEST["v1"] = dfTest

El conjunto de datos no cumple los criterios de evaluación, ya que no llega a 100 registros, sino a 91 (y con registros repetidos), y hay una notable diferencia entre el número de registros de REGISTRAR_TOMA_MEDICAMENTO (28) y REGISTRAR_DESPEDIDA (9), por lo que el conjunto **no es válido**.

### 1.2. Conjunto v2

Tras la generación de nuevos diálogos por parte de los usuarios y la mejora de la herramienta que genera el conjunto de datos a partir de los diálogos para que elimine registros duplicados, se procede a analizar el nuevo conjunto de datos generado.

In [ ]:
DATASET_VERSION = "v2" # Versión del conjunto de datos

df = loadDataframe(DATASET_VERSION)
df.head()

In [ ]:
df.shape

In [ ]:
df["intent"].unique()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df[df['text']=='Hola']

In [ ]:
df['intent'].value_counts()

In [ ]:
filteredColumns = ['REGISTRAR_TOMA_MEDICAMENTO', 'REGISTRAR_ESTADO_EMOCIONAL', 'REGISTRAR_CITA_MEDICA', 'REGISTRAR_SALUDO', 'REGISTRAR_DESPEDIDA']
filter = df['intent'].isin(filteredColumns)

In [ ]:
dfFiltered = filterDataframe(df)
dfFiltered.head()

In [ ]:
dfFiltered.shape

In [ ]:
dfFiltered['intent'].value_counts()

In [ ]:
dfTrain, dfTest = saveTrainTestDataset(DATASET_VERSION)
DATASETS["v2"] = df
DATATRAIN["v2"] = dfTrain
DATATEST["v2"] = dfTest

El conjunto de datos cumple los criterios de evaluación, supera los 100 registros y está balanceado, por lo tanto, **es válido**.

## 2. Entrenamiento de modelos

In [ ]:
import os
from google.cloud import dialogflow

PROJECT_ID = "***REMOVED***"
SESSION = "123456789"
LANGUAGE_CODE = "es"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jovyan/credentials/google/key.json"

def identifyIntentWithDialogflow(text):
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(PROJECT_ID, SESSION)
    text_input = dialogflow.TextInput(text=text, language_code=LANGUAGE_CODE)
    query_input = dialogflow.QueryInput(text=text_input)
    response = session_client.detect_intent(
        request={"session": session, "query_input": query_input}
    )
    intent = response.query_result.intent.display_name
    return intent


In [ ]:
def getValidator(languajeModel):
    if languajeModel == "dialogflow":
        return identifyIntentWithDialogflow

def validate(test, languajeModel):
    validator = getValidator(languajeModel)
    total = 0
    success = 0
    intents = ['REGISTRAR_TOMA_MEDICAMENTO', 'REGISTRAR_ESTADO_EMOCIONAL', 'REGISTRAR_CITA_MEDICA', 'REGISTRAR_SALUDO', 'REGISTRAR_DESPEDIDA']
    for item in test:
        expectedIntent = item["intent"]
        if expectedIntent in intents:
            # En Google Action, el intent NO_IDENTIFICADO es el intent por defecto, que 
            # conservamos con el nombre que Google le da, por legibilidad.
            if "NO_IDENTIFICADO" == expectedIntent:
                expectedIntent = "Default Fallback Intent"
            text = item["text"]
            receivedIntent = validator(text)
            if expectedIntent == receivedIntent:
                success += 1
            else:
                print(f"FAILED: expected: {expectedIntent}, received: {receivedIntent} on text: {text}")
            total += 1
    accuracy = round(success/total*100, 2)
    return total, success, accuracy

### 2.1. Modelo Dialogflow: Dataset v1

### 2.2. Modelo Dialogflow: Dataset v2

In [2]:
DATASET_VERSION = "v2"
LANGUAJE_MODEL = "dialogflow"
# Versión del conjunto de datos a utilizar: v2
%run -i dialogflow.py "v2"

-----------
DIALOGFLOW GENERATE
-----------
Dataset version: v2
Language model: dialogflow
Generate entities
Generate intents
/home/jovyan/data/dialogflow/v2/intents
Generate agent
/home/jovyan/data/dialogflow/v2/agent.json
Generate package
/home/jovyan/data/dialogflow/v2/package.json


A continuación:

- Eliminar todos los intents del proyecto de Dialogflow, excepto el "DefaultFallbackIntent".
- Importar en el proyecto de Dialogflow el dataset generado, comprimido en .zip (no añadir model_info.json).
- Eliminar el intent "NO_IDENTIFICADO", puesto que contiene textos que, por ahora, no han sido clasificados, de modo que deberían ir a "DefaultFallbackIntent".

In [ ]:
test = getDataset(DATASET_VERSION, "_test")
total, success, accuracy = validate(test, LANGUAJE_MODEL)

print(f"Total: {total}, success: {success}, accuracy: {accuracy}%")

El modelo **es válido**, puesto que el criterio de evaluación exige una tasa de acierto superior al 85%, y este modelo ha alcanzado un **89.66%**.